In [7]:
# Dependencies
import requests
import json
import pandas as pd

from sqlalchemy import create_engine
# Google developer API key
from config import gkey

In [8]:
accidents_csv = "../../Resources/raw/Saferparks-dataset-legacy-v2.csv"
table_name = "accident_data"

geocode_json_base = "https://maps.googleapis.com/maps/api/geocode/json?"

In [9]:
df = pd.read_csv(accidents_csv, encoding="ANSI")

In [10]:
unique_cities = df["acc_city"].unique()

city_state_df = df[["acc_state", "acc_city"]].loc[df["acc_city"].isin(unique_cities)]
city_state_df.drop_duplicates(inplace=True)
city_state_df.reset_index(inplace=True, drop=True)
city_state_df.head()

,acc_state,acc_city
0,WI,Green Bay
1,WI,Madison
2,WI,Wisconsin Dells
3,WA,Puyallup
4,CA,Anaheim


In [87]:
city_state_df.loc[city_state_df["acc_city"]=="Fairfield"]


,acc_state,acc_city
238,IL,Fairfield
356,NJ,Fairfield


In [13]:

city_lat = []
city_lng = []
for i in range(len(city_state_df)):
    params = {
        "address": f'{city_state_df.loc[i,"acc_city"]}, {city_state_df.loc[i,"acc_state"]}',
        "key": gkey
    }
    geo_data = requests.get(geocode_json_base, params).json()
    try:
        city_lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
        city_lng.append(geo_data["results"][0]["geometry"]["location"]["lng"])
        
        print(f'{params["address"]} : {i}')
    except:
        print(f'Address: {params["address"]} not found : {i}')
        city_lat.append("null")
        city_lng.append("null") 


Green Bay, WI : 0
Madison, WI : 1
Wisconsin Dells, WI : 2
Puyallup, WA : 3
Anaheim, CA : 4
Naples, FL : 5
Buena Park, CA : 6
Los Angeles, CA : 7
Riverside, CA : 8
Upland, CA : 9
San Dimas, CA : 10
San Diego, CA : 11
Irvine, CA : 12
Palm Springs, CA : 13
Phillips, WI : 14
West Allis, WI : 15
Lake Gregory, CA : 16
Escondido, CA : 17
San Jose, CA : 18
Carlsbad, CA : 19
Milwaukee, WI : 20
Vista, CA : 21
Santa Barbara, CA : 22
Angels Camp, CA : 23
Universal City, CA : 24
Aurora, CO : 25
Santa Monica, CA : 26
nan, FL : 27
Egg Harbor Twp., NJ : 28
Orlando, FL : 29
Dania Beach, FL : 30
Stuttgart, AR : 31
Cherry Hill, NJ : 32
nan, IL : 33
Jacksonville, FL : 34
Santa Clara, CA : 35
nan, KY : 36
nan, OK : 37
Jackson, NJ : 38
Marlton, NJ : 39
Bayonne, NJ : 40
Vallejo, CA : 41
Tulsa, OK : 42
nan, TX : 43
nan, OH : 44
Sewell, NJ : 45
Gilroy, CA : 46
nan, PA : 47
Pleasanton, CA : 48
Sacramento, CA : 49
Seaside Heights, NJ : 50
Silver Springs, FL : 51
Garfield, NJ : 52
Roseville, CA : 53
Toms River, N

In [19]:
city_df = pd.DataFrame({
    "acc_city": city_state_df["acc_city"],
    "acc_state": city_state_df["acc_state"],
    "city_lat": city_lat,
    "city_lng": city_lng
})
city_df = city_df[city_df.acc_city != "null"]
city_df.dropna(inplace=True)
city_df.head()

,acc_city,acc_state,city_lat,city_lng
0,Green Bay,WI,44.5133,-88.0133
1,Madison,WI,43.0731,-89.4012
2,Wisconsin Dells,WI,43.6275,-89.771
3,Puyallup,WA,47.1854,-122.293
4,Anaheim,CA,33.8366,-117.914


In [20]:
city_df.loc[city_df["acc_city"] == "Fairfield"]

,acc_city,acc_state,city_lat,city_lng
74,Fairfield,NJ,40.8837,-74.306
447,Fairfield,IL,38.3789,-88.3598


In [17]:
acc_states = df["acc_state"].unique()
state_lat = []
state_lng = []
for state in acc_states:
    params = {
        "address": state,
        "key": gkey
    }
    geo_data = requests.get(geocode_json_base, params).json()
    try:
        state_lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
        state_lng.append(geo_data["results"][0]["geometry"]["location"]["lng"])
        print(state)
    except:
        print(f'address: {city} not found')
        state_lat.append("null")
        state_lng.append("null") 

WI
WA
CA
FL
CO
NJ
AR
IL
KY
OK
TX
OH
PA
NY
MO
NV
MA
GA
IA
VA
MD
MI
WV
NC
TN
AZ
CT
IN
RI
LA


In [18]:
state_df = pd.DataFrame({
    "acc_state": acc_states,
    "state_lat": state_lat,
    "state_lng": state_lng
})

state_df.head()

,acc_state,state_lat,state_lng
0,WI,43.784440,-88.787868
1,WA,47.751074,-120.740139
2,CA,36.778261,-119.417932
3,FL,27.664827,-81.515754
4,CO,37.062500,-95.677068


In [21]:
df = pd.merge(df, city_df, how="left", on=["acc_city", "acc_state"])
df = pd.merge(df, state_df, how="left", on="acc_state")

In [22]:
df.drop_duplicates(subset="acc_id", inplace=True)
df.head()

,acc_id,acc_date,acc_state,acc_city,fix_port,source,bus_type,industry_sector,device_category,device_type,...,report,category,mechanical,op_error,employee,notes,city_lat,city_lng,state_lat,state_lng
0,1005755,9/12/2009,WI,Green Bay,F,"Wisconsin Dept. of Commerce, Safety and Buildi...",Amusement park,recreation,play equipment,Slide,...,NaN,Impact: hit something in participatory attraction,NaN,NaN,NaN,NaN,44.5133,-88.0133,43.784440,-88.787868
1,1005754,6/28/2009,WI,Madison,P,Media Report,Unknown,amusement ride,spinning,Ferris/gondola wheel,...,NaN,Equipment failure,1.0,1.0,NaN,http://wisinfo.biz/ic/caspio/amusementpark.html,43.0731,-89.4012,43.784440,-88.787868
2,1005753,5/30/2009,WI,Wisconsin Dells,F,Media Report,Amusement park,recreation,go-kart,Go-kart,...,NaN,Collision: patron-controlled vehicles,NaN,NaN,NaN,http://wisinfo.biz/ic/caspio/amusementpark.html,43.6275,-89.771,43.784440,-88.787868
3,1004578,4/17/2009,WA,Puyallup,P,Washington Dept. of Labor and Industries,Carnival or rental,amusement ride,spinning,Swing ride,...,NaN,Equipment failure,1.0,NaN,NaN,http://www.king5.com/article/news/local/amusem...,47.1854,-122.293,47.751074,-120.740139
4,1007272,4/5/2009,CA,Anaheim,F,California Division of Occupational Safety and...,Amusement park,amusement ride,cars & track rides,Track ride,...,NaN,Illness or neurological symptoms,NaN,NaN,NaN,NaN,33.8366,-117.914,36.778261,-119.417932


In [23]:
df.loc[df["acc_city"] == "Fairfield"]

,acc_id,acc_date,acc_state,acc_city,fix_port,source,bus_type,industry_sector,device_category,device_type,...,report,category,mechanical,op_error,employee,notes,city_lat,city_lng,state_lat,state_lng
492,916217,8/12/2007,NJ,Fairfield,P,New Jersey Dept. of Community Affairs,Carnival or rental,recreation,inflatable,Inflatable slide,...,NaN,Impact: hit something in participatory attraction,NaN,NaN,NaN,Inadvertent Rider Mishap,40.8837,-74.306,40.058324,-74.405661
1010,916054,5/5/2007,NJ,Fairfield,P,New Jersey Dept. of Community Affairs,Carnival or rental,recreation,inflatable,Inflatable game,...,NaN,Fall: in climb or play area,NaN,NaN,NaN,Loss of Control by Rider,40.8837,-74.306,40.058324,-74.405661
1180,914382,12/3/2006,NJ,Fairfield,P,New Jersey Dept. of Community Affairs,Carnival or rental,recreation,inflatable,Inflatable slide,...,NaN,Collision: patrons collided (participatory),NaN,NaN,NaN,Rider Hit by Others,40.8837,-74.306,40.058324,-74.405661
1279,914373,9/22/2006,NJ,Fairfield,P,New Jersey Dept. of Community Affairs,Carnival or rental,recreation,inflatable,Inflatable obstacle course,...,NaN,Fall: in climb or play area,NaN,NaN,NaN,Inadvertent Rider Mishap,40.8837,-74.306,40.058324,-74.405661
1304,914358,9/9/2006,NJ,Fairfield,P,New Jersey Dept. of Community Affairs,Carnival or rental,recreation,inflatable,Inflatable bouncer,...,NaN,Collision: patrons collided (participatory),NaN,NaN,NaN,Rider Hit by Others,40.8837,-74.306,40.058324,-74.405661
1677,914936,7/19/2006,NJ,Fairfield,P,New Jersey Dept. of Community Affairs,Carnival or rental,recreation,challenge activity,Mechanical bull-type,...,NaN,Fall: ejection/fall from ride,NaN,NaN,NaN,Inadvertent Rider Mishap,40.8837,-74.306,40.058324,-74.405661
1910,914933,6/24/2006,NJ,Fairfield,P,New Jersey Dept. of Community Affairs,Carnival or rental,recreation,inflatable,Inflatable obstacle course,...,NaN,Fall: in climb or play area,NaN,NaN,NaN,Inadvertent Rider Mishap,40.8837,-74.306,40.058324,-74.405661
2062,914858,6/2/2006,NJ,Fairfield,P,New Jersey Dept. of Community Affairs,Carnival or rental,recreation,inflatable,Inflatable slide,...,NaN,Impact: hit something in participatory attraction,1.0,NaN,NaN,Product failure,40.8837,-74.306,40.058324,-74.405661
2248,915815,4/2/2006,NJ,Fairfield,P,New Jersey Dept. of Community Affairs,Carnival or rental,recreation,challenge activity,Climbing wall,...,NaN,Fall: in climb or play area,NaN,1.0,NaN,"Operator error, Loss of Control by Rider",40.8837,-74.306,40.058324,-74.405661
2344,914828,1/22/2006,NJ,Fairfield,F,New Jersey Dept. of Community Affairs,Family entertainment center,recreation,play equipment,Play structure,...,NaN,Impact: hit something in participatory attraction,NaN,NaN,NaN,Inadvertent Rider Mishap,40.8837,-74.306,40.058324,-74.405661


In [24]:
# Create Database Connection
# ----------------------------------
# Creates a connection to our DB
engine = create_engine("sqlite:///../../Resources/park_accidents.sqlite", encoding="ANSI")
conn = engine.connect()

In [25]:
df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()